In [1]:
import sys
import datasets
from dotenv import dotenv_values
from pathlib import Path

config = dotenv_values("./../../config/.env") # take environment variables from .env.
base_path = Path(config["BASE_PATH"])
sys.path.append(str(base_path/"code"))
from ZeDiAc.Annotators import ActiveAnnotator

iteration = 2

data = datasets.load_from_disk(base_path/f"data/processed/active-learning-iteration-{iteration}")

In [48]:
annotator = ActiveAnnotator(
    data = data,
    labels = {f"loss_AL_{iteration}": "loss", f"unexpected_AL_{iteration}": "unexpected"},
    out_file = base_path/f"data/labeling/active-learning-iteration-{iteration}.pkl",
    text_column = "text",
    batch_size = 25,
    target_label = f"unexpected_AL_{iteration}",
    target_true_id = 0
)

In [ ]:
annotator.label_batch(strategy="lc")

Unexpected quirks

exceptional != unexpected
forward looking statements

In [74]:
annotator.data.strategy.value_counts()

lc_unexpected_AL_2      250
lc_unexpected_AL_1      200
lc_loss_AL_1            200
lc_unexpected           200
hc_t_unexpected         100
lc_loss                 100
hc_t_loss               100
lc_loss_AL_2            100
hc_t_unexpected_AL_2    100
hc_t_loss_AL_1           50
hc_t_loss_AL_2           50
hc_t_unexpected_AL_1     50
Name: strategy, dtype: int64

In [75]:
annotator.get_annotations_count_per_label()

,loss,unexpected,labeled_examples
count,451,183,1500


In [ ]:
print("unexpected")
print((annotator.data.p_unexpected > 0.5).sum())
print((annotator.data.p_unexpected_AL_1 > 0.5).sum())
print((annotator.data.p_unexpected_AL_2 > 0.5).sum())
print("loss")
print((annotator.data.p_loss > 0.5).sum())
print((annotator.data.p_loss_AL_1 > 0.5).sum())
print((annotator.data.p_loss_AL_2 > 0.5).sum())

In [35]:
print(((annotator.data.p_loss > 0.5)&(annotator.data.p_unexpected > 0.5)).sum())
print(((annotator.data.p_loss_AL_1 > 0.5)&(annotator.data.p_unexpected_AL_1 > 0.5)).sum())
print(((annotator.data.p_loss_AL_2 > 0.5)&(annotator.data.p_unexpected_AL_2 > 0.5)).sum())

1294
717
301


# Initialize the labeling pickle for NEW round

In [24]:
import datasets
import pandas as pd
from scipy.special import softmax

iteration = 1

data_loss = datasets.load_from_disk(base_path/f"data/processed/loss/active-learning-iteration-{iteration}")

data_unexpected = datasets.load_from_disk(base_path/f"data/processed/unexpected/active-learning-iteration-{iteration}")
annotations = pd.read_pickle(Path(config["BASE_PATH"])/f"data/labeling/active-learning-iteration-{iteration-1}.pkl")

df = data_loss.to_pandas().merge(data_unexpected.to_pandas()[["report_id", "paragraph_nr", f"logits_unexpected_AL_{iteration}"]], on=["report_id", "paragraph_nr"])
df.drop(columns=["input_ids", "token_type_ids", "attention_mask", "text", "n_words", "filing_type"], inplace=True)
df[f"p_loss_AL_{iteration}"] = df[f"logits_loss_AL_{iteration}"].apply(lambda x: softmax(x)[0])
df[f"p_unexpected_AL_{iteration}"] = df[f"logits_unexpected_AL_{iteration}"].apply(lambda x: softmax(x)[0])

# Be careful with this line as it potentially overwrites already existing labels

In [ ]:
#df.merge(annotations, on=["report_id", "paragraph_nr"]).to_pickle(Path(config["BASE_PATH"])/f"data/labeling/active-learning-iteration-{iteration}.pkl")